In [47]:
import time
import matplotlib
from barMap import BarMap
matplotlib.use('TkAgg')  # Or 'Qt5Agg', 'Qt4Agg'
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import RadioButtons, Slider
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button
from matplotlib.widgets import TextBox

# Create a figure and axis
fig, ax = plt.subplots()
fig.set_size_inches(16,6)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add padding around the graph using subplots_adjust
plt.subplots_adjust(left=0.25)

# Create the slider axis and the slider object
barSpeed = Slider(plt.axes([0.05, 0.9, 0.1, 0.05]) , 'Speed', 1, 5.0, valinit=1, valstep=0.5)

ax_sbutton = plt.axes([0.025, 0.05, 0.05, 0.075])  # Position [left, bottom, width, height]
start_button = Button(ax_sbutton, 'Start')

ax_rbutton = plt.axes([0.1, 0.05, 0.05, 0.075])  # Position [left, bottom, width, height]
reset_button = Button(ax_rbutton, 'Reset')
# Number of bars
n_bars = 10
waitingTime = 1
nums = np.random.choice(range(1, 100), size=n_bars, replace=False)

def SetNumbersAndBars():
    ax.clear()
    global myBarMap, nums
    # X positions of the bars
    x = np.arange(n_bars)
    # Heights of the bars
    nums = np.random.choice(range(1, 100), size=n_bars, replace=False)
    # Create the bar plot
    bars = ax.bar(x, nums)
    barPatches = bars.patches

# dictionary, value to bar rendered
    barDictionary = {}
    for height, patch in zip(nums, barPatches):
        patch.set_color('blue') #default color
        barDictionary[height] = patch

    myBarMap = BarMap(barDictionary)

    # Set up axis params
    ax.set_xlim(-1, n_bars) 
    ax.set_ylim(0, max(nums) + 2)
    ax.set_xticks(x)
    ax.set_xticklabels(nums)

    #The numbers
    print("Numbers")
    for i in range(0,len(nums)):
        print(f'{nums[i]}, ', end='')

SetNumbersAndBars()

selectedAlgo = 'quick' #the first radio option

stepSet = False
def step(event):
    global stepSet
    stepSet = True

def updateSpeed(expression):
    global barSpeed, barSpeedTextBox
    if expression.isdigit():
        barSpeed = int(expression)

def Reset(event):
    SetNumbersAndBars()
    start_button.set_active(True)
    start_button.label.set_color('black')
    plt.draw()
    pass

def Start(event):
    global selectedAlgo
    start_button.set_active(False)
    start_button.label.set_color('grey')
    reset_button.set_active(False)
    reset_button.label.set_color('grey')
    plt.draw()
    Render()
    if selectedAlgo == 'quick':
        print("Starting")
        QuickSort()
    elif selectedAlgo == 'bubble':
        print("Starting")
        BubbleSort()

def FinalAnimation():
    for n in  nums:
        myBarMap.SetColorForValue(n, 'green')
        Render()
    reset_button.set_active(True)
    reset_button.label.set_color('black')

def Partition(array, low, high):
    pivot = array[high]
    i = low -1

    myBarMap.SetColorForValue(pivot, 'red')
    myBarMap.SetColorForValue(array[i], 'yellow') 

    Render()
    time.sleep(0.5 / barSpeed.val)

    for j in range(low,high):
        myBarMap.SetColorForValue(array[j], 'yellow') 
        Render()
        time.sleep(0.5 / barSpeed.val)
        if array[j] <= pivot:
            i = i+1
            myBarMap.AnimateSwap(array[i], array[j])
            array[i], array[j] = array[j], array[i]

            while not myBarMap.IsIdle():
                myBarMap.Update(barSpeed.val)
                Render()

            myBarMap.SetColorForValue(array[i], 'blue') 
            Render()
            time.sleep(0.5 / barSpeed.val)
            
        myBarMap.SetColorForValue(nums[j], 'blue')           
        if j > 0:
            myBarMap.SetColorForValue(nums[j-1], 'blue')   
            Render()
            time.sleep(0.5 / barSpeed.val)
        
    myBarMap.SetColorForValue(array[i+1], 'yellow') 
    myBarMap.SetColorForValue(array[high], 'yellow') 
    Render()
    time.sleep(0.5 / barSpeed.val)
    myBarMap.AnimateSwap(array[i+1], array[high])    
    array[i+1], array[high] = array[high], array[i+1]

    while not myBarMap.IsIdle():
        myBarMap.Update(barSpeed.val)
        Render() 
    
    myBarMap.SetColorForValue(pivot, 'blue')
    myBarMap.SetColorForValue(array[i+1], 'blue')
    myBarMap.SetColorForValue(array[high], 'blue')
    Render()
    time.sleep(0.5 / barSpeed.val)
    return i+1

def QuickSortR(array, low, high):
    if low < high:
        pi = Partition(array, low, high)
        QuickSortR(array, low, pi - 1)
        QuickSortR(array, pi + 1, high)

def QuickSort():
    global nums
    QuickSortR(nums, 0, len(nums) -1)
    FinalAnimation()


def BubbleSort():
    for j in range(0,len(nums) -1):
        for i in range(0,len(nums) -j -1):
            myBarMap.SetColorForValue(nums[i], 'yellow')
            myBarMap.SetColorForValue(nums[i+1], 'yellow')
            if i > 0:
                myBarMap.SetColorForValue(nums[i-1], 'blue')

            Render()
            time.sleep(0.5 / barSpeed.val)

            if nums[i] > nums[i+1]:

                myBarMap.AnimateSwap(nums[i], nums[i+1])
                nums[i], nums[i+1] = nums[i+1], nums[i]   
                i += 1
                
            while not myBarMap.IsIdle():
                myBarMap.Update(barSpeed.val)
                Render()

            time.sleep(0.5 / barSpeed.val)

            myBarMap.SetColorForValue(nums[i], 'blue')
            if i <  len(nums) -1:          
                myBarMap.SetColorForValue(nums[i+1], 'blue')
                
            if i > 0:
                myBarMap.SetColorForValue(nums[i-1], 'blue')
                    
            Render()

    FinalAnimation()
                
def Render():
    fig.canvas.draw()
    fig.canvas.flush_events()
    time.sleep(0.0016)
    ax.set_xticklabels(nums)

algorithms = {
    'QuickSort': QuickSort,
    'BubbleSort': BubbleSort,
}

#Some GUI
# Add the button to the plot
# Callback for the radio button selection
def radioChange(label):
    global selectedAlgo
    print(f'Radio change {label}')
    selectedAlgo = label



start_button.on_clicked(Start)
reset_button.on_clicked(Reset)

ax_radio = plt.axes([0.05, 0.2, 0.1, 0.2])  # Position [left, bottom, width, height]
radio = RadioButtons(ax_radio, ('quick', 'bubble'))
radio.on_clicked(radioChange)
plt.show()




Numbers
75, 85, 58, 45, 3, 13, 13, 83, 20, 22, Numbers
35, 12, 69, 66, 49, 17, 97, 31, 48, 86, Numbers
22, 79, 41, 65, 72, 80, 3, 72, 52, 66, Numbers
23, 53, 29, 77, 11, 48, 68, 45, 53, 71, Numbers
74, 31, 33, 64, 78, 4, 16, 76, 86, 30, Numbers
94, 40, 6, 59, 64, 89, 15, 48, 16, 31, Radio change bubble
Starting
Numbers
66, 50, 40, 32, 85, 93, 3, 88, 37, 6, Numbers
22, 37, 6, 28, 8, 48, 38, 63, 78, 88, Numbers
36, 46, 64, 6, 99, 37, 15, 66, 57, 42, Numbers
66, 4, 52, 43, 28, 71, 9, 24, 71, 10, Radio change quick
Starting
Numbers
68, 1, 46, 28, 14, 77, 9, 73, 97, 51, Starting
Numbers
2, 81, 14, 41, 15, 48, 36, 81, 56, 82, Numbers
84, 84, 71, 67, 26, 57, 74, 18, 44, 99, Numbers
70, 5, 47, 35, 48, 8, 15, 51, 74, 52, 